# 0.3 Napari

## Set up

In [1]:
%run ./0.2-jvs-before_to_start.ipynb

## Libraries

In [2]:
from cell_viewer.UJ.UJExperiment import UJExperiment
from pprint import pprint
import napari

## Load experiment data

### Experiment directory

In [3]:
experiment_dir = data_raw_dir(
    "uJ_data",
    "pBGT-AMP-Pulse",
    "20190325_Bruno_pBGT_quimiostato"
)
experiment_dir.exists() and experiment_dir.is_dir()

True

### Read into UJExperiment object

In [4]:
experiment = UJExperiment(
    name=experiment_dir.name,
    path=experiment_dir
)

experiment.name, experiment.path.exists()

('20190325_Bruno_pBGT_quimiostato', True)

## Identify experiment traps

In [5]:
experiment_traps = experiment.trap_data("xy*")
experiment_traps.keys()

dict_keys(['xy01', 'xy02', 'xy03', 'xy11', 'xy12', 'xy13', 'xy14', 'xy20', 'xy21', 'xy22', 'xy23', 'xy25', 'xy26', 'xy27', 'xy28', 'xy29', 'xy30', 'xy33', 'xy34', 'xy35', 'xy36', 'xy37', 'xy39', 'xy40', 'xy41', 'xy42', 'xy43', 'xy44', 'xy45', 'xy46', 'xy47', 'xy05', 'xy06', 'xy07', 'xy08', 'xy09', 'xy10', 'xy15', 'xy16', 'xy17', 'xy18', 'xy19', 'xy24', 'xy31', 'xy32', 'xy38', 'xy04'])

## Read tifs from specific trap

In [6]:
xy_tifs = (
    experiment_traps
    .get("xy02")
    .read_tifs(search_on = ["data_tif"], compute = False)
)

xy_tifs

{'data_tif': {'DIC': dask.array<concatenate, shape=(25, 512, 640), dtype=uint16, chunksize=(1, 512, 640), chunktype=numpy.ndarray>,
  'DsRed': dask.array<concatenate, shape=(25, 512, 640), dtype=uint16, chunksize=(1, 512, 640), chunktype=numpy.ndarray>,
  'GFP': dask.array<concatenate, shape=(25, 512, 640), dtype=uint16, chunksize=(1, 512, 640), chunktype=numpy.ndarray>}}

## Create CellViewerWidgets

In [7]:
import datetime
import numpy
from enum import Enum
from pathlib import Path
from magicgui import magicgui

class Medium(Enum):
    """Using Enums is a great way to make a dropdown menu."""
    Glass = 1.520
    Oil = 1.515
    Water = 1.333
    Air = 1.0003

@magicgui(
    call_button="Calculate",
    layout="vertical",
    result_widget=True,
    # numbers default to spinbox widgets, but we can make
    # them sliders using the `widget_type` option
    slider_float={"widget_type": "FloatSlider", "max": 100},
    slider_int={"widget_type": "Slider", "readout": False},
    radio_option={
        "widget_type": "RadioButtons",
        "orientation": "horizontal",
        "choices": [("first option", 1), ("second option", 2)],
    },
    filename={"label": "Pick a file:"},  # custom label
)
def widget_demo(
    boolean=True,
    integer=1,
    spin_float=3.14159,
    slider_float=43.5,
    slider_int=550,
    string="Text goes here",
    dropdown=Medium.Glass,
    radio_option=2,
    date=datetime.date(1999, 12, 31),
    time=datetime.time(1, 30, 20),
    datetime=datetime.datetime.now(),
    filename=Path.home(),  # path objects are provided a file picker
):
    """Run some computation."""
    return locals().values()

## Run napari

In [8]:
%gui qt5
viewer = napari.Viewer()

for directory, tifs in xy_tifs.items():
    for name, tif in tifs.items():

        if "gfp" in name.lower():
            colormap = "green"
            opacity = 1.0
        elif "red" in name.lower():
            colormap = "red"
            opacity = 1.0
        else:
            colormap = "gray"
            opacity = 0.1

        viewer.add_image(
            data=tif.compute(),
            name=f"{directory}_{name}",
            contrast_limits=[0, 2000],
            multiscale=False,
            blending="additive",
            colormap=colormap,
            opacity=opacity
        )
        
viewer.window.add_dock_widget(widget_demo, area="right")